In [ ]:
import os
import pandas as pd
import csv
import time

In [ ]:
# 화면 가로 확장 코드 (기본 width 50%)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

### 전력+기온 결합 데이터 폴더 생성

In [ ]:
try:
    os.mkdir('./Combined_Data')
    for y in range(2013, 2017):
        os.mkdir('./Combined_Data/'+str(y))
    print('폴더 생성 완료.')
except FileExistsError:
    print('해당 폴더가 이미 존재합니다.\n폴더를 확인해주세요.')

### 데이터 결합 코드

In [ ]:
print('------------------------------< Start Working!! >------------------------------\n')

month = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}

for y in range(2013, 2017):
    
    for m in range(1,13):
        csv_electric = pd.read_csv("./Electric_Data/Electirc_Data_CSV_FInal/"+str(y)+"/"+str(y)+'_'+str(m)+'_'+"House1_Ch1_Electric_Data_Final.csv", encoding='cp949')
        csv_weather = pd.read_csv("./Weather_Data/Weather_Data_24H_CSV/"+str(y)+"/"+str(y)+'_'+str(m)+'_'+"London_Temperature_24H.csv", encoding='utf-8')
        csv_CPI = pd.read_csv('./CPI.csv')
        csv_electric['Temp']=csv_weather['Temp']
        csv_electric['Humidity']=csv_weather['Humidity']
        csv_electric['CPI']=''
        date=str(y)+'_'+str(m)
        
        cpi=float(csv_CPI['CPI'].loc[csv_CPI['Date']==date])
        
        s = 0
        for d in range(month[m]):
            d=d+1 
            
            for n in range(s, s+24):
                combine_list=[]
                combine_list.append(csv_electric['DateTime'][n])
                combine_list.append(csv_electric['Graph_Date'][n])
                combine_list.append(csv_electric['Day'][n])
                combine_list.append(csv_electric['Holiday'][n])
                combine_list.append(csv_electric['Seq'][n])
                combine_list.append(csv_electric['Watt'][n])
                combine_list.append(csv_electric['Temp'][n])
                combine_list.append(csv_electric['Humidity'][n])
                combine_list.append(cpi)
                with open("./Combined_Data/"+str(y)+"/"+str(y)+'_'+str(m)+'_'+"House1_Ch1_Combined_Data.csv", 'a', newline='') as file:
                                writer = csv.writer(file)
                                writer.writerow(combine_list)
            s=s+24
            
        csv_origin = pd.read_csv("./Combined_Data/"+str(y)+"/"+str(y)+'_'+str(m)+'_'+"House1_Ch1_Combined_Data.csv", names = ['DateTime', 'Graph_Date', 'Day', 'Holiday', 'Seq', 'Watt', "Temp('C)", 'Humidity(%)', 'CPI'], header = None, encoding='cp949')
        csv_origin.to_csv("./Combined_Data/"+str(y)+"/"+str(y)+'_'+str(m)+'_'+"House1_Ch1_Combined_Data.csv", encoding='cp949')
        
print('\n------------------------------< Job Done!! >------------------------------\n')

### 1년 단위 통합 CSV 저장

In [ ]:
for y in range(2013, 2017):
    folders = os.listdir('./Combined_Data/'+str(y)+'/')

    df_year = pd.DataFrame()
    for m in range(0,len(folders)):
        m=m+1
        file = './Combined_Data/'+str(y)+'/'+str(y)+'_'+str(m)+'_'+'House1_Ch1_Combined_Data.csv'
        df_m = pd.read_csv(file, encoding='cp949')
        df_year = pd.concat([df_year, df_m])
        
    df_year = df_year.drop([df_year.columns[0]], axis=1)
    df_year = df_year.reset_index(drop=True)
    df_year.to_csv("./Combined_Data/"+str(y)+'_'+"House1_Ch1_Combined_Data_Y.csv", encoding='cp949')

print('\n------------------------------< Year Merge Done!! >------------------------------\n')

### 데이터 합본 저장

#### 3년치

In [ ]:
df_3Y = pd.DataFrame()
for y in range(2013, 2016):
    file = './Combined_Data/'+str(y)+'_'+'House1_Ch1_Combined_Data_Y.csv'
    df_y= pd.read_csv(file, encoding='cp949')
    df_3Y = pd.concat([df_3Y, df_y])
    
df_3Y = df_3Y.drop([df_3Y.columns[0]], axis=1)
df_3Y = df_3Y.reset_index(drop=True)
df_3Y.to_csv("./Combined_Data/House1_Ch1_Combined_Data_13-15.csv", encoding='cp949')

In [ ]:
Data_3Y = pd.read_csv('./Combined_Data/House1_Ch1_Combined_Data_13-15.csv', encoding='cp949')
Data_3Y.info()

#### 4년치

In [ ]:
df_4Y = pd.DataFrame()
for y in range(2013, 2017):
    file = './Combined_Data/'+str(y)+'_'+'House1_Ch1_Combined_Data_Y.csv'
    df_y= pd.read_csv(file, encoding='cp949')
    df_4Y = pd.concat([df_4Y, df_y])
    
df_4Y = df_4Y.drop([df_4Y.columns[0]], axis=1)
df_4Y = df_4Y.reset_index(drop=True)
df_4Y.to_csv("./Combined_Data/House1_Ch1_Combined_Data_13-16.csv", encoding='cp949')

In [ ]:
Data_4Y = pd.read_csv('./Combined_Data/House1_Ch1_Combined_Data_13-16.csv', encoding='cp949')
Data_4Y.insert(3,'Unix_Time','')

for i in range(len(Data_4Y)):
    Y=int(Data_4Y['Graph_Date'][i][:4])
    M=int(Data_4Y['Graph_Date'][i][5:7])
    D=int(Data_4Y['Graph_Date'][i][8:10])
    date = datetime.datetime(Y, M, D)
    unix=int(time.mktime(date.timetuple())/1000) # 시,분,초 정보는 0으로 일괄처리 되기 때문에 1000으로 나눠서 뒤 3자리수는 제거
    Data_4Y['Unix_Time'][i]=unix

In [ ]:
Data_4Y.info()

In [ ]:
Data_2013=pd.read_csv('./Combined_Data/2013_House1_Ch1_Combined_Data_Y.csv', encoding='cp949')
Data_2014=pd.read_csv('./Combined_Data/2014_House1_Ch1_Combined_Data_Y.csv', encoding='cp949')
Data_2015=pd.read_csv('./Combined_Data/2015_House1_Ch1_Combined_Data_Y.csv', encoding='cp949')
Data_2016=pd.read_csv('./Combined_Data/2016_House1_Ch1_Combined_Data_Y.csv', encoding='cp949')

### 그래프 확인

#### plt 그래프 함수

In [ ]:
def Watt_Graph(name, x, y, year):
    font_head = {'color': 'k', 'weight': 'bold', 'size': 20}
    font_body = {'color': 'k', 'weight': 'bold', 'size': 15}
    plt.figure(figsize=(30,10))
    plt.plot(x, y, '.-', linewidth=1, color='y')
    plt.tick_params(axis='x', colors='k', size=10, labelsize=15)
    plt.tick_params(axis='y', colors='k', size=10, labelsize=15)
    plt.xlabel("Time", labelpad=15, fontdict=font_body)
    plt.ylabel('Watt', labelpad=15, fontdict=font_body)
    plt.title(name, fontdict=font_head, pad=20)
    plt.margins(x=0.02)

    if year==1:
        plt.xticks([0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334], labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    if year==3:
        plt.xticks([0, 365, 730], labels=['2013', '2014', '2015'])
        
    plt.show

In [ ]:
def Weather_Graph(name1, name2, x, y1, y2, year):
    plt.figure(figsize=(30,20))
    font_head = {'color': 'k', 'weight': 'bold', 'size': 20}
    font_body = {'color': 'k', 'weight': 'bold', 'size': 15}

    # 1번째 그래프
    ax1 = plt.subplot(2, 1, 1)
    ax1.spines['left'].set_color('#ff7f0e')
    ax1.spines['left'].set_lw(2)
    ax1.spines['bottom'].set_color('#ff7f0e')
    ax1.spines['bottom'].set_lw(2)

    ax1.tick_params(axis='x', colors='k', size=10, labelsize=15)
    ax1.tick_params(axis='y', colors='k', size=10, labelsize=15)

    plt.plot(x, y1, '.-', color='#ff7f0e', linewidth=1)
    plt.title(name1, fontdict=font_head, pad=20)
    plt.xticks(visible=False)
    plt.ylabel("Temp('C)",labelpad=15, fontdict=font_body)
    
    # 2번째 그래프
    ax2 = plt.subplot(2, 1, 2, sharex=ax1)
    ax2.spines['left'].set_color('royalblue')
    ax2.spines['left'].set_lw(2)
    ax2.spines['bottom'].set_color('royalblue')
    ax2.spines['bottom'].set_lw(2)

    ax2.tick_params(axis='x', colors='k', size=10, labelsize=15)
    ax2.tick_params(axis='y', colors='k', size=10, labelsize=15)

    plt.plot(x, y2, '.-', color='#1f77b4', linewidth=1)
    plt.title(name2, fontdict=font_head, pad=20)
    plt.xlabel('Time',labelpad=15, fontdict=font_body)
    plt.ylabel('Humidity(%)',labelpad=15, fontdict=font_body)
    
    plt.margins(x=0.02)
    
    if year==1:
        plt.xticks([0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334], labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
    if year==3:
        plt.xticks([0, 365, 730], labels=['2013', '2014', '2015'])

    plt.show()

In [ ]:
Watt_Graph("2013's Time-Watt Graph", Data_2013['Graph_Date'], Data_2013['Watt'], 1)
Weather_Graph("2013's Time-Temp Graph", "2013's Time-Humidity Graph", Data_2013['Graph_Date'], Data_2013["Temp('C)"], Data_2013['Humidity(%)'], 1)

In [ ]:
Watt_Graph("2014's Time-Watt Graph", Data_2014['Graph_Date'], Data_2014['Watt'], 1)
Weather_Graph("2014's Time-Temp Graph", "2014's Time-Humidity Graph", Data_2014['Graph_Date'], Data_2014["Temp('C)"], Data_2014['Humidity(%)'], 1)

In [ ]:
Watt_Graph("2015's Time-Watt Graph", Data_2015['Graph_Date'], Data_2015['Watt'], 1)
Weather_Graph("2015's Time-Temp Graph", "2015's Time-Humidity Graph", Data_2015['Graph_Date'], Data_2015["Temp('C)"], Data_2015['Humidity(%)'], 1)

In [ ]:
Watt_Graph("13-15's Time-Watt Graph", Data_3Y['Graph_Date'], Data_3Y['Watt'], 3)
Weather_Graph("13-15's Time-Temp Graph", "13-15's Time-Humidity Graph", Data_3Y['Graph_Date'], Data_3Y["Temp('C)"], Data_3Y['Humidity(%)'], 3)